In [22]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt

In [23]:
transformations = transforms.Compose([
    transforms.ToTensor()
])

In [24]:
batch_size = 64
num_classes = 10
num_channels = 1
img_size = 28*28
patch_size = 7
num_patches = (img_size / patch_size)*2
attention_heads = 4
embedding_dim = 16
transformer_blocks = 4
mlp_nodes = 64

In [25]:
train_dataset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transformations)
val_dataset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transformations)

In [26]:
train_data = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
val_data = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)

In [27]:
# Patch Embeddings - Part 1 of the ViT architecture

class PatchEmbedding(nn.Module):
    def __init__(self):
        super().__init__()
        self.patch_embed = nn.Conv2d(num_channels, embedding_dim, kernel_size=patch_size, stride=patch_size)

    def forward(self, x):
        x = self.patch_embed(x)
        x = x.flatten(2)
        x = x.transpose(1, 2)
        
        return x

In [28]:
# Transformer Encoder - Part 2 of the ViT archotecture

class TransformerEncoder(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer_norm1 = nn.LayerNorm(embedding_dim)
        self.mha = nn.MultiheadAttention(embedding_dim, attention_heads, batch_first=True)
        self.layer_norm2 = nn.LayerNorm(embedding_dim)
        self.mlp = nn.Sequential(
            nn.Linear(embedding_dim, mlp_nodes),
            nn.GELU(),
            nn.Linear(mlp_nodes, embedding_dim)
        )

    def forward(self, x):
        res1 = x
        x = self.layer_norm1(x)

        x = self.mha(x, x, x)[0] + res1
        res2 = x

        x = self.layer_norm2(x)
        x = self.mlp(x) + res2

        return x


In [29]:
# class for MLP Head for Classification - Part 3 of the ViT architecture

class MLP_Head(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer_norm = nn.LayerNorm(embedding_dim)
        self.mlp_head = nn.Sequential(
            nn.Linear(embedding_dim, num_classes)
        )

    def forward(self, x):
        x = self.layer_norm(x)
        x = self.mlp_head(x)

        return x

In [30]:
class VisionTransformer(nn.Module):
    def __init__(self):
        super().__init__()
        self.patch_embedding = PatchEmbedding()
        self.cls_token = nn.Parameter(torch.randn(1, 1, embedding_dim))
        self.position_embedding = nn.Parameter(torch.randn(1, (img_size // patch_size) ** 2 + 1, embedding_dim))
        self.transformer_blocks = nn.Sequential(*[TransformerEncoder() for _ in range (transformer_blocks)])
        self.mlp_head = MLP_Head()

    def forward(self, x):
        x = self.patch_embedding(x)
        
        B = x.size(0)
        cls_tokens = self.cls_token.expand(B, -1, -1)
        x = torch.cat((cls_tokens, 1), dim=1)
        
        x = x + self.position_embedding
        
        x = self.transformer_blocks(x)
        x = x[:, 0]
        x = self.mlp_head(x)
        
        return x

In [31]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = VisionTransformer().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss()

In [32]:
for epoch in range(5):
    model.train()
    total_loss = 0
    correct_epoch = 0
    total_epoch = 0
    print(f"\nEpoch {epoch+1}")

    for batch_idx, (images, labels) in enumerate(train_data):
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)

        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss+=loss.item()
        preds = outputs.argmax(dim=1)

        correct = (preds == labels).sum().item()
        accuracy = 100.0 * correct / labels.size(0)

        correct_epoch += correct
        total_epoch += labels.size(0)

        if batch_idx % 100 == 0:
            print(f"  Batch {batch_idx+1:3d}: Loss = {loss.item():.4f}, Accuracy = {accuracy:.2f}%")

    epoch_acc = 100.0 * correct_epoch / total_epoch
    print(f"==> Epoch {epoch+1} Summary: Total Loss = {total_loss:.4f}, Accuracy = {epoch_acc:.2f}%")


Epoch 1


TypeError: expected Tensor as element 1 in argument 0, but got int